In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

from loguru import logger as logging

from pcb.hamlib import open_hamiltonian_file
from pcb.qiskit import to_evolution_gate

# maxcut Hamiltonian -> Ising
# FILE = "out/ham/binaryoptimization__maxcut__random__ham-graph-complete_bipart.hdf5.zip"
# KEY = "complbipart-n-100_a-50_b-50"

# max3sat Hamiltonian -> 3SAT
FILE = "out/ham/binaryoptimization__max3sat__satlib__lran-ham.hdf5.zip"
KEY = "0-f1000.cnf-900-res"

start = datetime.now()
with open_hamiltonian_file(FILE) as fp:
    # print(len(fp.keys()))
    # for k in fp.keys():
    #     print(k, len(to_evolution_gate(fp[k][()]).operator))
    gate = to_evolution_gate(fp[KEY][()])
logging.info("Loaded in {}", datetime.now() - start)
logging.info("Number of terms: {}", len(gate.operator))

2025-02-05 17:35:07.008 | INFO     | __main__:<module>:22 - Loaded in 0:00:01.615525
2025-02-05 17:35:07.010 | INFO     | __main__:<module>:23 - Number of terms: 13548


In [3]:
from itertools import combinations

from qiskit.quantum_info import SparsePauliOp
from tqdm.notebook import tqdm

from pcb.reordering.utils import term_groups


def check_coloring(coloring: dict[int, list[int]], operator: SparsePauliOp):
    start = datetime.now()
    terms = operator.to_list()
    color: dict[int, int] = {}  # inversion of coloring
    for c, v in coloring.items():
        for i in v:
            color[i] = c
    for q, grp in tqdm(list(term_groups(operator).items())):
        cs = set(color[i] for i in grp)
        if len(cs) == len(grp):
            continue
        logging.error("Problem at group of qubit {}", q)
        for i1, i2 in combinations(grp, 2):
            if color[i1] != color[i2]:
                continue
            col = color[i1]
            s1, s2 = terms[i1][0][::-1], terms[i2][0][::-1]
            for qubit, (p1, p2) in enumerate(zip(s1, s2)):
                if not (p1 == "I" or p2 == "I"):
                    s1_, s2_ = s1.replace("I", "_"), s2.replace("I", "_")
                    a = " " + (" " * qubit) + "↓"
                    raise RuntimeError(
                        f"{col=} {i1=} {i2=} {qubit=}\n{a}\n>{s1_}<\n>{s2_}<"
                    )
    logging.info("Checked coloring in {}", datetime.now() - start)

In [4]:
from datetime import datetime

from loguru import logger as logging

from pcb.reordering import degree_reordering

start = datetime.now()
gate_r, coloring = degree_reordering(gate)
logging.info("Reordered operator in {}", datetime.now() - start)
logging.info("Used {} colors", len(coloring))

check_coloring(coloring, gate.operator)

2025-02-05 17:35:12.656 | INFO     | __main__:<module>:9 - Reordered operator in 0:00:05.294983
2025-02-05 17:35:12.659 | INFO     | __main__:<module>:10 - Used 67 colors


  0%|          | 0/900 [00:00<?, ?it/s]

2025-02-05 17:35:14.189 | INFO     | __main__:check_coloring:33 - Checked coloring in 0:00:01.529023


In [5]:
from datetime import datetime

from loguru import logger as logging

from pcb.reordering import saturation_reordering

start = datetime.now()
gate_r, coloring = saturation_reordering(gate)
logging.info("Reordered operator in {}", datetime.now() - start)
logging.info("Used {} colors", len(coloring))

check_coloring(coloring, gate.operator)

2025-02-05 17:35:36.258 | INFO     | __main__:<module>:9 - Reordered operator in 0:00:22.006913
2025-02-05 17:35:36.260 | INFO     | __main__:<module>:10 - Used 66 colors


  0%|          | 0/900 [00:00<?, ?it/s]

2025-02-05 17:35:37.786 | INFO     | __main__:check_coloring:33 - Checked coloring in 0:00:01.524871


In [6]:
from datetime import datetime

from loguru import logger as logging

from pcb.reordering import misra_gries_reordering
from pcb.reordering.utils import is_ising

if is_ising(gate):
    start = datetime.now()
    gate_r, coloring = misra_gries_reordering(gate)
    logging.info("Reordered operator in {}", datetime.now() - start)
    logging.info("Used {} colors", len(coloring))

    check_coloring(coloring, gate.operator)

In [7]:
from datetime import datetime

from loguru import logger as logging

from pcb.reordering import simplicial_reordering
from pcb.reordering.utils import is_3sat

if is_3sat(gate):
    start = datetime.now()
    gate_r, coloring = simplicial_reordering(gate)
    logging.info("Reordered operator in {}", datetime.now() - start)
    logging.info("Used {} colors", len(coloring))

    check_coloring(coloring, gate.operator)

2025-02-05 17:35:54.275 | INFO     | __main__:<module>:11 - Reordered operator in 0:00:13.895527
2025-02-05 17:35:54.278 | INFO     | __main__:<module>:12 - Used 66 colors


  0%|          | 0/900 [00:00<?, ?it/s]

2025-02-05 17:35:55.965 | INFO     | __main__:check_coloring:33 - Checked coloring in 0:00:01.686573
